In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import string
from spacy.lang.en.stop_words import STOP_WORDS
import spacy

In [17]:
data = pd.read_csv('amazon_data.txt', sep='\t', header=None)

In [18]:
data.columns = ["Review", "Sentiment"]
data.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [19]:
data["Sentiment"].value_counts() # checking data

1    500
0    500
Name: Sentiment, dtype: int64

In [20]:
data.isnull().sum() # checking for null values

Review       0
Sentiment    0
dtype: int64

In [25]:
## helpers for data cleaning
punctuation = string.punctuation
stop_words = list(STOP_WORDS)
nlp = spacy.load("en_core_web_sm")
numbers = string.digits
# print(stop_words)
## function that cleans input text
def cleaning_function(input_text):
    text = nlp(input_text)
    tokens = []
    for token in text:
        temp = token.lemma_.lower()
        tokens.append(temp)

    cleaned_tokens = []
    for token in tokens:
        if token not in stop_words and token not in punctuation and token not in numbers:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [26]:
## test
cleaned_text = cleaning_function(data["Review"][4])
print(cleaned_text)

['mic', 'great']
['being', 'whereafter', 'two', 'only', 'thus', 'through', 'further', 'hereupon', 'would', 'one', 'without', 'are', 'whole', 'why', 'rather', 'several', 'any', 'next', 'had', 'might', '’m', 'and', 'quite', 'an', 'eleven', 'latter', 'used', 'enough', 'the', 'nobody', 'so', 'were', 'whither', 'however', 'latterly', 'yours', 'both', 'mostly', 'been', 'nothing', 'bottom', 'too', 'somewhere', 'during', 'her', 'itself', 'three', 'onto', 'sometimes', 'toward', 'something', 'never', 'in', 'hereafter', 'must', 'thereby', 'former', 'by', 'from', 'fifteen', 'amongst', 'after', 'together', 'herein', 'if', 'before', 'give', 'here', 'twenty', 'back', 'noone', 'five', 'others', 'even', 'that', 'empty', 'full', 'a', 'they', 'upon', 'always', 'along', 'about', 'moreover', 'each', 'else', 'see', 'for', 'n‘t', 'perhaps', 'around', 'your', 'get', 'has', 'yourselves', '’d', 'four', 'many', 'already', 'can', 'more', 'his', 'show', 'on', 'whether', 'indeed', '’re', "'s", 'although', 'besides'